In [24]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [25]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [26]:
problem = get_problem('kakuro')

pprint.pprint(problem)

board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

print('Input puzzle')
#pretty_print(board, h_const, v_const)

# This does not solve the problem, just makes it the same as the input
solution = board

# We print the solution in a way to make it easy to debug and check
#print('Solution')
#pretty_print(solution, h_const, v_const)

answer = {
    'board': board,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
# Note: We are doing this just to get used to it in following exercises mostly
submit_answer('kakuro', answer)

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', 'X', 'X', '.', '.'],
           ['X', '.', '.', '.', '.'],
           ['X', '.', '.', '.', '.'],
           ['X', '.', '.', 'X', 'X']],
 'h_constraints': [[5], [13], [14], [11]],
 'id': 2,
 'v_constraints': [[23], [7], [6], [7]]}
Input puzzle
Ignore: not checking values yet


In [27]:
# Code Block 4: Representation of the problem

def initialize_domains(board):
    return {(row, col): list(range(1, 10)) for row in range(len(board)) for col in range(len(board[0])) if board[row][col] == '.'}

def initialize_constraints(board, h_const, v_const):
    constraints = {}
    # Populate the constraints dictionary with horizontal constraints
    for r, row in enumerate(h_const):
        sum_constraint = row[0]
        cells_in_block = [(r+1, c) for c, value in enumerate(board[r+1]) if value == '.']
        for cell in cells_in_block:
            constraints[cell] = constraints.get(cell, (0, 0))
            constraints[cell] = (sum_constraint, constraints[cell][1])

    # Populate the constraints dictionary with vertical constraints
    for c, col in enumerate(v_const):
        sum_constraint = col[0]
        cells_in_block = [(r, c+1) for r, value in enumerate(board) if value[c+1] == '.']
        for cell in cells_in_block:
            constraints[cell] = constraints.get(cell, (0, 0))
            constraints[cell] = (constraints[cell][0], sum_constraint)
    return constraints


In [28]:
#Code block 5: Constraint Propagation


# ensure that if a number is already assigned to a cell within a block, it cannot be assigned to another cell in the same block
def apply_unique_constraint(domains, constraints, cells_in_block):
    """Apply the unique values constraint to the domains of the cells in a block."""
    block_values = set()
    for cell in cells_in_block:
        if len(domains[cell]) == 1:
            block_values.add(domains[cell][0])
    for cell in cells_in_block:
        if len(domains[cell]) > 1:
            domains[cell] = [value for value in domains[cell] if value not in block_values]

# Code Block 5 (Continued): Implementing apply_sum_constraint

from itertools import combinations

# Precompute all possible combinations for each sum
possible_combinations = {}
for r in range(2, 10):  # From 2 to 9 inclusive
    for s in range(r, 45):  # Possible sum
        possible_combinations[(r, s)] = [combo for combo in combinations(range(1, 10), r) if sum(combo) == s]

def apply_sum_constraint(domains, constraints, cells_in_block, sum_constraint):
    """Apply the sum constraint to the domains of the cells in a block."""
    if len(cells_in_block) == 1:
        # If there's only one cell in the block, the domain is the sum itself if it's between 1 and 9.
        cell = cells_in_block[0]
        if 1 <= sum_constraint <= 9:
            domains[cell] = [sum_constraint] if sum_constraint in domains[cell] else []
        else:
            domains[cell] = []
        return

    # Get all possible combinations for the sum and number of cells in the block
    possible_sums = possible_combinations.get((len(cells_in_block), sum_constraint), [])
    
    # Intersect the possible sums with the current domains
    for i, cell in enumerate(cells_in_block):
        valid_values = set(val[i] for val in possible_sums)
        domains[cell] = [value for value in domains[cell] if value in valid_values]

def propagate_constraints(domains, constraints):
    """Apply constraints to reduce the domains of the cells."""
    for cell, (h_sum, v_sum) in constraints.items():
        h_cells = [c for c in constraints if c[0] == cell[0] and constraints[c][0] == h_sum]
        v_cells = [c for c in constraints if c[1] == cell[1] and constraints[c][1] == v_sum]
        apply_unique_constraint(domains, constraints, h_cells)
        apply_unique_constraint(domains, constraints, v_cells)
        apply_sum_constraint(domains, constraints, h_cells, h_sum)
        apply_sum_constraint(domains, constraints, v_cells, v_sum)

# Call the constraint propagation function to reduce the domains before starting the backtracking process.
propagate_constraints(domains, constraints)


In [29]:
# Code Block 6: Backtracking with a Stack (Modified with Debugging)

from copy import deepcopy

def is_valid_assignment(domains, constraints, cell, value):
    # Debugging: Print the attempt to assign a value
    print(f"Attempting to assign value {value} to cell {cell}")
    
    # Check if the value is unique in its block.
    row, col = cell
    h_sum, v_sum = constraints[cell]
    
    # Check horizontal block
    h_block_cells = [c for c in constraints if c[0] == row and constraints[c][0] == h_sum]
    if any(domains[c] == [value] for c in h_block_cells if c != cell):
        print(f"Value {value} already in horizontal block for cell {cell}")  # Debugging
        return False
    
    # Check vertical block
    v_block_cells = [c for c in constraints if c[1] == col and constraints[c][1] == v_sum]
    if any(domains[c] == [value] for c in v_block_cells if c != cell):
        print(f"Value {value} already in vertical block for cell {cell}")  # Debugging
        return False
    
    # Check if the value does not violate the sum constraint.
    if not is_sum_valid(domains, h_block_cells, h_sum, cell, value):
        print(f"Value {value} violates horizontal sum constraint for cell {cell}")  # Debugging
        return False
    
    if not is_sum_valid(domains, v_block_cells, v_sum, cell, value):
        print(f"Value {value} violates vertical sum constraint for cell {cell}")  # Debugging
        return False
    
    return True

def select_unassigned_variable(domains, constraints):
    # Debugging: Print the process of selecting an unassigned variable
    print("Selecting the next unassigned variable using MRV heuristic")
    
    mrv_cell = None
    min_domain_size = float('inf')
    
    for cell, values in domains.items():
        if len(values) > 1 and len(values) < min_domain_size:
            min_domain_size = len(values)
            mrv_cell = cell
    
    if mrv_cell is None:
        print("No unassigned variables found.")  # Debugging
    else:
        print(f"Selected cell {mrv_cell} with domain size {min_domain_size}")  # Debugging
    
    return mrv_cell

def backtrack(domains, constraints):
    stack = [(deepcopy(domains), [])]
    backtrack_count = 0
    
    while stack:
        current_domains, assignments = stack.pop()
        if all(len(values) == 1 for values in current_domains.values()):
            return assignments, backtrack_count
        
        cell = select_unassigned_variable(current_domains, constraints)
        if cell is None:
            print("No variable selected for assignment. Possible dead end.")  # Debugging
            continue
        
        for value in current_domains[cell]:
            if is_valid_assignment(current_domains, constraints, cell, value):
                new_domains = deepcopy(current_domains)
                new_domains[cell] = [value]
                new_assignments = assignments + [(cell, value)]
                stack.append((new_domains, new_assignments))
                print(f"Assigned value {value} to cell {cell}")  # Debugging
            else:
                backtrack_count += 1
                print(f"Backtracking: Value {value} not valid for cell {cell}")  # Debugging
    
    return None, backtrack_count


In [30]:
# Code Block 7: Testing the Kakuro Solver (Modified with Debugging)

# Fetch a new Kakuro problem
problem = get_problem('kakuro')
print('Input puzzle:')
pprint.pprint(problem)

# Extract the board and constraints from the problem
board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

# Initialize domains and constraints for the Kakuro puzzle
domains = initialize_domains(board)
constraints = initialize_constraints(board, h_const, v_const)

# Apply constraint propagation to reduce the domains
propagate_constraints(domains, constraints)

# Debugging: Print domains after constraint propagation
print("Domains after constraint propagation:")
for cell, possible_values in domains.items():
    print(f"Cell {cell}: {possible_values}")

# Run the backtracking algorithm to find a solution
solution, backtrack_count = backtrack(domains, constraints)
print(f"Backtracking count: {backtrack_count}")

# If a solution is found, format it appropriately
if solution:
    print("Solution found:")
    for cell, value in solution:
        board[cell[0]][cell[1]] = value
    pretty_print(board, h_const, v_const)
    answer = {
        'board': board,
        'h_const': h_const,
        'v_const': v_const,
        'id': problem['id']
    }
    # Submit the answer to the server
    submit_answer('kakuro', answer)
else:
    print("No solution found.")


Input puzzle:
{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', '.', '.', '.']],
 'h_constraints': [[13], [10], [21], [18]],
 'id': 1,
 'v_constraints': [[4], [11], [30], [17]]}
Domains after constraint propagation:
Cell (1, 1): [1]
Cell (1, 2): []
Cell (1, 3): [6]
Cell (2, 1): []
Cell (2, 2): [2]
Cell (2, 3): [7]
Cell (3, 2): []
Cell (3, 3): [8]
Cell (3, 4): []
Cell (4, 2): [5]
Cell (4, 3): []
Cell (4, 4): []
Selecting the next unassigned variable using MRV heuristic
No unassigned variables found.
No variable selected for assignment. Possible dead end.
Backtracking count: 0
No solution found.
